# 💡 Homography and Perspective Transform in OpenCV

`Homography` allows you to **map points from one plane to another**, which is more general than `affine transformations`. It preserves lines but can handle perspective distortions.

---

## 🔹 Mathematical Intuition

The homography matrix `H` is a 3x3 matrix that transforms a point `(x, y)` in the source image to `(x', y')` in the destination image:

$$ \begin{bmatrix} x' \\ y' \\ w' \end{bmatrix} = H \begin{bmatrix} x \\ y \\ 1 \end{bmatrix}, \quad H = \begin{bmatrix} h_{11} & h_{12} & h_{13} \\ h_{21} & h_{22} & h_{23} \\ h_{31} & h_{32} & 1 \end{bmatrix} $$

After dividing by `w'`:
$$ x' = x'/w', \quad y' = y'/w' $$

- Homography is `projective`, it can handle perspective effects.
- Affine is a special case (last row `[0, 0, 1]`).
- We need at least `4 point correspondences` to solve for H.

In [ ]:
# Imports and Setup
import cv2
import numpy as np
import os
from tools.tools import LearnTools

learn_tools = LearnTools()
%matplotlib inline

In [ ]:
# Load image
# img_url = "https://i.ibb.co/5x276TvQ/1.jpg"
# img_url = "https://i.ibb.co/QjkCQ6Vm/2.jpg"
# img_url = "https://i.ibb.co/NyT8LB5/test.jpg"
img_url = "https://i.ibb.co.com/Pv7CFM89/196ef9c8-fe1c-4a13-8774-6078761fd220.jpg"


if os.path.exists('testImage.jpg'):
    image = cv2.imread('testImage.jpg')
else:
    pil_image = await learn_tools.get_image(img_url=img_url, padding=0)
    pil_image.save('testImage.jpg', 'JPEG')
    image = learn_tools.pil_to_cv2(pil_image)

# Display Rsult
learn_tools.show_multiple_images(
        image_plotting_data=[
            {'title': 'Original Image', 'image': image}
        ]
    )

## 🔹 Define Corresponding Points

To compute a `homography`, you must provide at least 4 pairs of corresponding points between:
- The source image (`input perspective`)
- The destination plane (`rectified or target perspective`)

**Best practices:**
- Points must be geometrically consistent (same physical corners).
- Distribute points across the full object/region, not clustered.
- Avoid nearly collinear points — homography becomes unstable.
- Ensure a consistent order (e.g., top-left → top-right → bottom-right → bottom-left).

**For interactive point selection, OpenCV’s `cv2.setMouseCallback` can be used to collect coordinates directly from the image.**

## 🔹 Compute Homography
Use `cv2.findHomography()` to estimate the 3×3 projective transformation matrix `H`:
- With exact, manually selected points, a direct solution is sufficient.
- With noisy or automatically detected correspondences, enable `RANSAC` to:
    - Reject outliers
    - Improve robustness
    - Identify inliers via the returned status mask

`H, status = cv2.findHomography(src_points, dst_points, cv2.RANSAC, 3.0)`
- `H` → homography matrix
- `status` → binary mask indicating inlier correspondences

## 🔹 Apply Perspective Transform

Apply the homography to the entire image using `cv2.warpPerspective`:
- This maps the source image onto the destination plane defined by `poimnts_destination`.
- The output size must match the destination geometry, not the original image size.

`warped = cv2.warpPerspective(image, H, (dst_width, dst_height))`
- Using (`column`, `row`) is only correct if you explicitly want the original image dimensions.
- For document rectification or top-down views, always use the computed destination width and height.

In [ ]:
row, column = image.shape[:2]        # original image dimensions

points_source = np.float32([
    [25, 185],   # Top-left
    [152, 194],  # Top-right
    [142, 328],  # Bottom-right
    [20, 290]    # Bottom-left
])

# Width
width_top = np.linalg.norm(points_source[1] - points_source[0])
width_bottom = np.linalg.norm(points_source[2] - points_source[3])
max_width = int(max(width_top, width_bottom))

# Height
height_left = np.linalg.norm(points_source[3] - points_source[0])
height_right = np.linalg.norm(points_source[2] - points_source[1])
max_height = int(max(height_left, height_right))


points_destination = np.float32([
    [0, 0],
    [max_width - 1, 0],
    [max_width - 1, max_height - 1],
    [0, max_height - 1]
])


M = cv2.getPerspectiveTransform(points_source, points_destination)
warped = cv2.warpPerspective(image, M, (max_width, max_height))


img_vis = image.copy()
for pt in points_source:
    cv2.circle(img_vis, tuple(pt.astype(int)), 6, (0, 0, 255), -1)

learn_tools.show_multiple_images([
    {'title': 'Original Image with Source Points', 'image': img_vis},
    {'title': 'Perspective Corrected Output', 'image': warped}
])


## 🔹 Robust Homography with RANSAC

In real-world scenarios, point correspondences may have noise. **RANSAC** helps compute a robust homography:

- It iteratively selects random subsets of points.
- Finds the homography that maximizes the number of inliers.
- Minimizes the effect of outliers.

In [ ]:
H_ransac, status_ransac = cv2.findHomography(
    points_source,
    points_destination,
    cv2.RANSAC,
    ransacReprojThreshold=3.0
)

# Safety check
if H_ransac is None:
    raise RuntimeError("Homography computation failed")

# Warp using destination size (NOT original image size)
warped_ransac = cv2.warpPerspective(
    image,
    H_ransac,
    (max_width, max_height)
)


learn_tools.show_multiple_images([
    {'title': 'Perspective Corrected (RANSAC)', 'image': warped_ransac}
])
